In [1]:
%display latex
from dalgebra import *

# Non-trivial centralizers and the spectral curve

In this notebook we are going to explain and provide code on how we can get some non-trivial centralizers in differential algebra and, once a couple of commuting operators are found, we will compute the spectral curve for both.

This notebook will serve as a checking on the state on the repository and will push also the theory we have been developing further.

*Notation*: we will use the notation $(R,\partial)$ to described a differential ring and $R\{u\}$ as the ring of differential polynomials in $u$ (meaning, $R\{u\} = R[u,u',u'',\ldots]$ and all the derivations of $u$ are algebraically independent). A differential operator with coefficients over a differential ring $(R,\partial)$ is an element of the first Weyl algebra $R[\partial]$. This is a non-commutative ring where $\partial r = r' + r\partial$.

#### References

This work is based on the following references (will increase when needed):

1. *Computing Almost-commuting Basis of Ordinary Differential Operators*, A. Jiménez-Pastor, S. Rueda, M.A. Zurro, in the 48th International Symposium on Symbolic and Algebraic Computation (**short communication**), 2023.
2. *A *REDUCE* program for the integration of diferential polynomials*, A.H. Bilge in **Computer Physics Communications** (73-3), pp. 263-268, 1992.
3. *Towards the classification of scalar nonpolynomial evolution equations: Quasilinearity*, A.H. Bilge in **Computers and Mathematics with Applications** (49-11,12), pp. 1837-1848, 2005.
4. *Additive normal forms and integration of differential fractions*, F. Boulier, F. Lemaire, J. Lallemand, G. Regensburger, M. Rosenkranz, in **Journal of Symbolic Computation** (77), pp. 16-38, 2016.
5. *Factorization of KdV Schrödinger operators using differential subresultants*, J.J. Morales-Ruíz, S. Rueda, M.A. Zurro in **Advances in Applied Mathematics** (120), pp. 102065, 2020.
6. *Factoring Third Order Ordinary Differential Operators over Spectral Curves*, S. Rueda, M.A. Zurro in **ArXiV** [arXiv:2102.04733v1](https://www.arxiv.org/abs/2102.04733v1)

## Differential resultant of two ordinary differential operators

Let $(\mathbb{K}, \partial)$ be a differential field of characterisfic zero and consider two differential operators
$\mathcal{A}, \mathcal{B} \in \mathbb{K}[D]$, where for any $\alpha in \mathbb{K}$:

$$D \alpha = \alpha D + \partial(\alpha).$$

We want to compute the differential resultant of these two operators. This can be done algebraically, since these operators are basically polynomials in the non-commutative variable $D$, using Sylvester resultant.

Currently this structure is not available on the package `dalgebra` and the package `ore_algebra` may not work in all cases to handle these type of operators. 

Anyway, we can reinterpret differential operators as homogeneous differential polynomials in one variable (let call this variable $w(x)$). Then the differential resultant of the two differential operators is the differential resultant of the two differential polynomials eliminating the variable $w(x)$.


#### 1. Setting up the example

In the paper *Algebro-geometric solitonic solutions and Differential Galois Theory*, the authors studied some spectral curves. They considered the Schr\"odinger operator
$$L = -\partial^2  + u,$$
where $u(x)$ is a differential indeterminate that is calle a *potential*. 

Then, they consider some other operators (coming from the KdV hierarchy?):
$$\begin{array}{l}
    \tilde{P_1} = P_1,\\
    \tilde{P_3} = P_3 + c_1P_1,\\
    \tilde{P_5} = P_5 + c_2P_3 + c_1P_1,
\end{array}$$
where the operators $P_1$, $P_3$ and $P_5$ are defined as follows:
$$\begin{array}{l}
    P_1 = \partial,\\
    P_3 = -\partial^3+\frac{3}{2} u \partial +\frac{3}{4}u',\\
    P_5 = \partial^5-\frac{5}{2}u \partial^3-\frac{15}{4}u'\partial^2+\frac{15}{8}u^2\partial-\frac{25}{8}u''\partial -\frac{15}{16}u'''+\frac{15}{8}u u'.
\end{array}$$

The differential resultants they computed were:
$$dres(L - \lambda, \tilde{P}_3 - \mu),\qquad dres(L - \lambda, \tilde{P}_5 - \mu).$$

Hence we have to work on the differential ring: $\mathbb{C}\{u\}[c_i][\lambda, \mu]$ and consider those operators in that ring. We are going to create several variables for each step of the process:

* `B` will represent the ring with the potential: $\mathbb{C}\{u\}$.
* `R` will represent the addition of the constants: $\mathbb{C}\{u\}[c_i]$.
* `S` will represent the addition of the two spectral parameters: $\mathbb{C}\{u\}[c_i][\lambda,\mu]$.
* Finally, `F` will represent the ring with the extra differential variable $w(x)$ that allow us to represent the differential operators as differential polynomials: $\mathbb{C}\{u\}[c_i][\lambda, \mu]\{w\}$.

We are going to rearrange the creation of the ring so the differential computations are performed properly.

For doing so, we are going to compute the ring $\mathbb{C}[c_1,c_2,\lambda,\mu]\{u,w\}$. Then we can set the derivation on the base polynomial ring to be the constant zero (since all the variables there are constant w.r.t. the derivation) and the differential behavior of $u$ and $w$ is preserved properly.

In [6]:
B.<c_1,c_2,l,m> = QQ[]
dB = DifferentialRing(B, lambda p : 0)
F.<u,w> = DifferentialPolynomialRing(dB); F

Ring of operator polynomials in (u, w) over Differential Ring [[Multivariate Polynomial Ring in c_1, c_2, l, m over Rational Field], (0,)]

At this point we can create the differential operators as differential polynomials:

In [7]:
L = -w[2] + u[0]*w[0]

In [8]:
P1 = w[1]
P3 = -w[3] + (3/2)*u[0]*w[1] + (3/4)*u[1]*w[0]
P5 = w[5] - (5/2)*u[0]*w[3] - (15/4)*u[1]*w[2] + (15/8)*u[0]^2*w[1] - (25/8)*u[2]*w[1] - (15/16)*u[3]*w[0] + (15/8)*u[0]*u[1]*w[0]

In [9]:
tP3 = P3 + c_1*P1
tP5 = P5 + c_2*P3 + c_1*P1

In [10]:
op1 = L-l*w[0]; op2 = tP3 - m*w[0]; op3 = tP5 - m*w[0]

#### 2. The first differential resultant

We are going to look now to the differential resultant of `op1` and `op2`, i.e.:
$$dres(L-\lambda, \tilde{P}_3-\mu)$$
eliminating the variable $w$.

For doing so, we need to use the class `DifferentialSystem` indicating the two differential polynomials involved and the variables we will eliminate:

In [11]:
system1 = DifferentialSystem([op1, op2], variables=[w])
system1

System over [Ring of operator polynomials in (u, w) over Differential Ring [[Multivariate Polynomial Ring in c_1, c_2, l, m over Rational Field], (0,)]] with variables [(w_*,)]:
{
	u_0*w_0 - w_2 + (-l)*w_0 == 0
	3/4*u_1*w_0 + 3/2*u_0*w_1 - w_3 + c_1*w_1 + (-m)*w_0 == 0
}

In [13]:
dres_system1 = system1.diff_resultant(alg_res="macaulay")
dres_system1

1/4*u_0^3 + (-1/8)*u_2*u_0 + 1/16*u_1^2 + (c_1 + 3/4*l)*u_0^2 + (-1/4*c_1 - 1/4*l)*u_2 + (c_1^2 + c_1*l)*u_0 - c_1^2*l - 2*c_1*l^2 - l^3 - m^2

At this point, to extrac the coefficients we will need to change the order of the variables appearing in this polynomial:

**WARNING: Maybe this is something we can do automatically... however this comes from the fact that the original structure we wanted to create failed cuurently in `dalgebra`**

#### 3. The second differential resultant

We are going to look now to the differential resultant of `op1` and `op3`, i.e.:
$$dres(L-\lambda, \tilde{P}_5-\mu)$$
eliminating the variable $w$.

For doing so, we need to use the class `DifferentialSystem` indicating the two differential polynomials involved and the variables we will eliminate:

In [14]:
system2 = DifferentialSystem([op1, op3], variables=[w])
system2

System over [Ring of operator polynomials in (u, w) over Differential Ring [[Multivariate Polynomial Ring in c_1, c_2, l, m over Rational Field], (0,)]] with variables [(w_*,)]:
{
	u_0*w_0 - w_2 + (-l)*w_0 == 0
	15/8*u_1*u_0*w_0 + 15/8*u_0^2*w_1 + (-15/16)*u_3*w_0 + (-25/8)*u_2*w_1 + (-15/4)*u_1*w_2 + 3/4*c_2*u_1*w_0 + (-5/2)*u_0*w_3 + 3/2*c_2*u_0*w_1 + w_5 + (-c_2)*w_3 + c_1*w_1 + (-m)*w_0 == 0
}

In [16]:
dres_system2 = system2.diff_resultant(alg_res="macaulay")
dres_system2

9/64*u_0^5 + (-15/64)*u_2*u_0^3 + (3/8*c_2 + 15/64*l)*u_0^4 + 3/128*u_4*u_0^2 + (-3/64)*u_3*u_1*u_0 + 1/16*u_2^2*u_0 + 3/64*u_2*u_1^2 + (-13/32*c_2 - 5/16*l)*u_2*u_0^2 + (1/4*c_2^2 + 7/8*c_2*l + 5/8*l^2 + 3/4*c_1)*u_0^3 + (-1/128)*u_4*u_2 + (1/32*c_2 + 1/32*l)*u_4*u_0 + 1/256*u_3^2 + (-1/32*c_2 - 1/32*l)*u_3*u_1 + (1/32*c_2 + 1/64*l)*u_2^2 + (-1/8*c_2^2 - 3/4*c_2*l - 5/8*l^2 - 5/8*c_1)*u_2*u_0 + (1/16*c_2^2 - 1/4*c_2*l - 5/16*l^2 - 3/8*c_1)*u_1^2 + (3/4*c_2^2*l + 3/4*c_2*l^2 + c_1*c_2 + 1/4*c_1*l)*u_0^2 + (1/16*c_2*l + 1/16*l^2 + 1/16*c_1)*u_4 + (-1/4*c_2^2*l - 1/4*c_2*l^2 - 1/4*c_1*c_2)*u_2 + (c_1*c_2*l + c_1*l^2 + c_1^2)*u_0 - c_2^2*l^3 - 2*c_2*l^4 - l^5 - 2*c_1*c_2*l^2 - 2*c_1*l^3 - c_1^2*l - m^2

Once again, we reorganize the variable to be more readable with respect to the variables $\lambda$ and $\mu$: